In [1]:
## Initializations
%pip install transformers
%pip install SentencePiece

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Load Data
import pandas as pd
csv_file_path = 'init_input_all.csv'
sample_count = 0
data = pd.read_csv(csv_file_path, header=0)
print(data.columns)

Index(['ID', 'Type', 'Source'], dtype='object')


In [3]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [4]:
import torch
import math
## Translate data
language_source = 'en'
language_target_names = ['German', 'Greek', 'English', 'Spanish', 'Persian',
                         'French', 'Hindi', 'Croatian', 'Italian', 'Korean',
                         'Dutch', 'Romanian', 'Russian', 'Turkish', 'Chinese']
language_target_labels = ['de', 'el', 'en', 'es', 'fa', 'fr', 'hi', 'hr', 'it',
                          'ko', 'nl', 'ro', 'ru', 'tr', 'zh']
# language_target_names = ['Persian', 'Dutch']
# language_target_labels = ['fa', 'nl']


result_fw = []
hiddens_fw_avg = []
hiddens_fw_last = []
try:
  result_fw = pd.read_csv('temp_fw.csv', header=0)
  hiddens_fw_avg = torch.load("language_translate_fw_avg_output_sample"+str(sample_count)+".pth")
  hiddens_fw_last = torch.load("language_translate_fw_last_output_sample"+str(sample_count)+".pth")
  hiddens_fw_avg = hiddens_fw_avg.tolist()
  hiddens_fw_last = hiddens_fw_last.tolist()
  loaded_count_fw = min(len(result_fw) / (len(language_target_labels)-1),
                        len(hiddens_fw_avg) / (len(language_target_labels)-1),
                        len(hiddens_fw_last) / (len(language_target_labels)-1))
  print('%s %s %s' % (len(result_fw) / (len(language_target_labels)-1),
                      len(hiddens_fw_avg)  / (len(language_target_labels)-1),
                      len(hiddens_fw_last) / (len(language_target_labels)-1)))
  result_fw = result_fw.iloc[0:math.floor(loaded_count_fw) * (len(language_target_labels)-1)]
  hiddens_fw_avg = hiddens_fw_avg[0:math.floor(loaded_count_fw) * (len(language_target_labels)-1)]
  hiddens_fw_last = hiddens_fw_last[0:math.floor(loaded_count_fw) * (len(language_target_labels)-1)]
  result_fw = list(result_fw.values)
except Exception as e:
    print(e)
    result_fw = []
    hiddens_fw_avg = []
    hiddens_fw_last = []

    loaded_count_fw = 0
    print("Couldn't load checkpoint!")


for i, r in data.iterrows():
  if i < loaded_count_fw:
    continue
  label = r.iloc[1]
  sentence = r.iloc[2]
  for l_name, l_label in zip(language_target_names, language_target_labels):
    tokenizer.src_lang = 'en'
    if l_label != 'en':
      input_ids = tokenizer(sentence, return_tensors="pt").input_ids
      generated_tokens = model.generate(input_ids,
                                        forced_bos_token_id=tokenizer.get_lang_id(l_label),
                                        output_hidden_states=True)
      output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

      hidden_states = (model.get_encoder()(input_ids).last_hidden_state)
      hidden_avg = torch.mean(hidden_states, dim=1)[0,:]
      hidden_last = hidden_states[0,-1,:]
      hiddens_fw_avg.append(hidden_avg)
      hiddens_fw_last.append(hidden_last)
      hiddens_fw_avg_t = hiddens_fw_avg
      hiddens_fw_last_t = hiddens_fw_last
      result_fw.append([i, label, l_label, l_name, sentence, ''.join(output) ])
    else:
      output = sentence
    
    
  df_result_fw = pd.DataFrame(result_fw)
  df_result_fw.to_csv('temp_fw.csv', index=False, encoding='utf-8')
  try:
    hiddens_fw_avg_t = torch.stack([torch.tensor(arr) for arr in hiddens_fw_avg])
    hiddens_fw_last_t = torch.stack([torch.tensor(arr) for arr in hiddens_fw_last])
  except Exception as e:
    print(e)
    pass
    # print("Tensors already stacked!")
  torch.save(hiddens_fw_avg_t, "language_translate_fw_avg_output_sample"+str(sample_count)+".pth")
  torch.save(hiddens_fw_last_t, "language_translate_fw_last_output_sample"+str(sample_count)+".pth")
    
  print('%s: %s' % (i, sentence))

df_result_fw = pd.DataFrame(result_fw)
print(df_result_fw)

474.0 474.0 474.0
        0  1   2         3                                                  4  \
0       0  2  de    German  Andrei picked up the chair or the bag and the ...   
1       0  2  el     Greek  Andrei picked up the chair or the bag and the ...   
2       0  2  es   Spanish  Andrei picked up the chair or the bag and the ...   
3       0  2  fa   Persian  Andrei picked up the chair or the bag and the ...   
4       0  2  fr    French  Andrei picked up the chair or the bag and the ...   
...   ... ..  ..       ...                                                ...   
6631  473  0  nl     Dutch  Andrei held the bag and the chair; the chair w...   
6632  473  0  ro  Romanian  Andrei held the bag and the chair; the chair w...   
6633  473  0  ru   Russian  Andrei held the bag and the chair; the chair w...   
6634  473  0  tr   Turkish  Andrei held the bag and the chair; the chair w...   
6635  473  0  zh   Chinese  Andrei held the bag and the chair; the chair w...   

         

In [5]:
import torch
## Translate backward data



result_bw = []
hiddens_bw_avg = []
hiddens_bw_last = []
try:
  result_bw = pd.read_csv('temp_bw.csv', header=0)
  hiddens_bw_avg = torch.load("language_translate_bw_avg_output_sample"+str(sample_count)+".pth")
  hiddens_bw_last = torch.load("language_translate_bw_last_output_sample"+str(sample_count)+".pth")
  hiddens_bw_avg = hiddens_bw_avg.tolist()
  hiddens_bw_last = hiddens_bw_last.tolist()
  loaded_count_bw = min(len(result_bw),
                        len(hiddens_bw_avg),
                        len(hiddens_bw_last))
  print('%s %s %s' % (len(result_bw),
                      len(hiddens_bw_avg),
                      len(hiddens_bw_last)))
  result_bw = result_bw.iloc[0:math.floor(loaded_count_bw)]
  hiddens_bw_avg = hiddens_bw_avg[0:math.floor(loaded_count_bw)]
  hiddens_bw_last = hiddens_bw_last[0:math.floor(loaded_count_bw)]
  result_bw = list(result_bw.values)
except Exception as e:
    print(e)
    result_bw = []
    hiddens_bw_avg = []
    hiddens_bw_last = []

    loaded_count_bw = 0
    print("Couldn't load checkpoint!")
    
    
for i, r in df_result_fw.iterrows():
  if i < loaded_count_bw:
    continue
    
  label = r.iloc[1]
  sentence = r.iloc[5]
  l_name = r.iloc[3]
  l_label = r.iloc[2]

  tokenizer.src_lang = l_label
  if l_label != 'en':
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids
    generated_tokens = model.generate(input_ids,
                                      forced_bos_token_id=tokenizer.get_lang_id('en'),
                                      output_hidden_states=True)
    output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    hidden_states = (model.get_encoder()(input_ids).last_hidden_state)
    hidden_avg = torch.mean(hidden_states, dim=1)[0,:]
    hidden_last = hidden_states[0,-1,:]
    hiddens_bw_avg.append(hidden_avg)
    hiddens_bw_last.append(hidden_last)
  else:
    output = sentence
  result_bw.append([i, label, l_label, l_name, sentence, ''.join(output) ])

  df_result_bw = pd.DataFrame(result_bw)
  df_result_bw.to_csv('temp_bw.csv', index=False, encoding='utf-8')
  try:
    hiddens_bw_avg_t = torch.stack([torch.tensor(arr) for arr in hiddens_bw_avg])
    hiddens_bw_last_t = torch.stack([torch.tensor(arr) for arr in hiddens_bw_last])
  except Exception as e:
    print(e)
    pass
    # print("Tensors already stacked!")
  torch.save(hiddens_bw_avg_t, "language_translate_bw_avg_output_sample"+str(sample_count)+".pth")
  torch.save(hiddens_bw_last_t, "language_translate_bw_last_output_sample"+str(sample_count)+".pth")
    
  print('%s: %s' % (i, sentence))

df_result_bw = pd.DataFrame(result_bw)
print(df_result_bw)

5900 5900 5900


/scratch-local/bmehrparvar.5976626/ipykernel_3374531/1954630764.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hiddens_bw_avg_t = torch.stack([torch.tensor(arr) for arr in hiddens_bw_avg])
/scratch-local/bmehrparvar.5976626/ipykernel_3374531/1954630764.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hiddens_bw_last_t = torch.stack([torch.tensor(arr) for arr in hiddens_bw_last])


5900: Andrej se približio Jevgeniju; on se također približio Danju
5901: Andrei si avvicinò a Yevgeni; si avvicinò anche a Danny
5902: Andrei는 Yevgeni에 접근; 그는 또한 Danny에 접근
5903: Andrei benadert Yevgeni; hij benadert ook Danny
5904: Andrei s-a apropiat de Yevgeni; şi el s-a apropiat de Danny
5905: Андрей приблизился к Евгени; он также приблизился к Дани
5906: Andrei Yevgeni'ye yaklaştı; Danny'ye de yaklaştı
5907: 安德烈接近耶夫吉尼;他也接近丹尼。
5908: Jeder Stuhl wurde von jemandem abgelegt.
5909: Κάθε καρέκλα κατεβάστηκε από κάποιον
5910: Cada silla fue colocada por alguien.
5911: هر صندلی توسط کسی قرار گرفته است.
5912: Chaque fauteuil a été jeté par quelqu'un
5913: प्रत्येक कुर्सी को किसी ने छोड़ दिया था।
5914: Svaki stol je stavljen od strane nekoga.
5915: Ogni sedia è stata gettata da qualcuno.
5916: 각 좌석은 누군가에 의해 떨어졌다.
5917: Elke stoel werd door iemand geslagen.
5918: Fiecare scaun a fost aruncat de cineva
5919: Каждое кресло подняли кто-то.
5920: Her sandalye birileri tarafından yerleştirildi.
5

In [6]:
## Save output
import pandas as pd

df_result_fw.to_csv("language_translate_fw_output_sample"+str(sample_count)+".csv",
                 index=False, encoding='utf-8')
df_result_bw.to_csv("language_translate_bw_output_sample"+str(sample_count)+".csv",
                 index=False, encoding='utf-8')

try:
  hiddens_fw_avg = torch.stack(hiddens_fw_avg)
  hiddens_fw_last = torch.stack(hiddens_fw_last)
  hiddens_bw_avg = torch.stack(hiddens_bw_avg)
  hiddens_bw_last = torch.stack(hiddens_bw_last)
except Exception as e:
    print("Tensors already stacked!")

torch.save(hiddens_fw_avg, "language_translate_fw_avg_output_sample"+str(sample_count)+".pth")
torch.save(hiddens_fw_last, "language_translate_fw_last_output_sample"+str(sample_count)+".pth")
torch.save(hiddens_bw_avg, "language_translate_bw_avg_output_sample"+str(sample_count)+".pth")
torch.save(hiddens_bw_last, "language_translate_bw_last_output_sample"+str(sample_count)+".pth")

import zipfile

# List of files to be included in the zip file
files_to_zip = ["language_translate_fw_output_sample"+str(sample_count)+".csv",
                "language_translate_bw_output_sample"+str(sample_count)+".csv",
                "language_translate_fw_avg_output_sample"+str(sample_count)+".pth",
                "language_translate_fw_last_output_sample"+str(sample_count)+".pth",
                "language_translate_bw_avg_output_sample"+str(sample_count)+".pth",
                "language_translate_bw_last_output_sample"+str(sample_count)+".pth"]

# Path where the zip file will be created
zip_file_path = "language_translate_output_sample"+str(sample_count)+".zip"

# Create a new zip file
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    # Add each file to the zip file
    for file in files_to_zip:
        zipf.write(file)

print("Zip file created successfully:", zip_file_path)


Tensors already stacked!
Zip file created successfully: language_translate_output_sample0.zip
